In [1]:
#default_exp traintransfer

### set `data.yaml`

In [7]:
import yaml,os

In [8]:
os.environ['YAML_DATA']="/home/turgutluk/Vent_Seg_Project/dev/configs/data.yaml"
os.environ['YAML_TL']="/home/turgutluk/Vent_Seg_Project/dev/configs/transfer_learning.yaml"

In [9]:
with open(os.environ.get('YAML_DATA', '../data.yaml')) as f: data_config = yaml.load(f.read(), yaml.FullLoader)
with open(os.environ.get('YAML_TL', 'transfer_learning.yaml')) as f: tl_config = yaml.load(f.read(), yaml.FullLoader)

In [ ]:
data_config

In [10]:
tl_config

{'BRAIN': {'MR': {'TL_Brain_MR_Baseline_1': 'best_of_ATLAS_Brain_MR_Baseline_1',
   'TL_Brain_MR_Baseline_2': 'best_of_ATLAS_Brain_MR_Baseline_2',
   'TL_Brain_MR_Baseline_3': 'best_of_ATLAS_Brain_MR_Baseline_3',
   'TL_Brain_MR_Baseline_4': 'best_of_ATLAS_Brain_MR_Baseline_4',
   'TL_Brain_MR_Baseline_5': 'best_of_ATLAS_Brain_MR_Baseline_5',
   'TL_Brain_MR_Baseline_6': 'best_of_ATLAS_Brain_MR_Baseline_6',
   'TL_Brain_MR_Baseline_7': 'best_of_ATLAS_Brain_MR_Baseline_7',
   'TL_Brain_MR_Baseline_8': 'best_of_ATLAS_Brain_MR_Baseline_8',
   'TL_Brain_MR_Baseline_9': 'best_of_ATLAS_Brain_MR_Baseline_9',
   'TL_Brain_MR_Baseline_10': 'best_of_ATLAS_Brain_MR_Baseline_10',
   'TL_Brain_MR_Baseline_11': 'best_of_ATLAS_Brain_MR_Baseline_11'},
  'CT': {'TL_Brain_CT_Baseline_1': 'best_of_ATLAS_Brain_MR_Baseline_1',
   'TL_Brain_CT_Baseline_2': 'best_of_ATLAS_Brain_MR_Baseline_2',
   'TL_Brain_CT_Baseline_3': 'best_of_ATLAS_Brain_MR_Baseline_3',
   'TL_Brain_CT_Baseline_4': 'best_of_ATLAS_Brain_

## Weakly Supervised Transfer Learning Training Script

In [11]:
#export
from fastai2.notebook.core import *
import sys, os

# add local/ package to python path to allow script to access py modules
if not IN_NOTEBOOK: sys.path.insert(0, os.path.abspath("."))

In [12]:
#export
from fastai2.vision.all import *
from fastai2.data.all import *
from local.datasource import *
from local.models import *
from fastai2.torch_core import *
from fastai2.basics import *
from local.trainutils import *
from fastai2.callback.all import *
from fastai2.distributed import *
from time import time
import yaml

In [13]:
len(datasource_dict), datasource_dict # all possible dataset for experiments, (atlas, mr, ct) x (brain, ventricle)

(6,
 {'notl_brain_mr': functools.partial(<function create_mr_ct_dsource at 0x7f5ac68b3f80>, '/home/turgutluk/data/ventricles_data/mr', '/home/turgutluk/data/ventricles_data/csvs/mr_splits_df.csv', <function tfm_image_pt at 0x7f5b396a6200>, <function tfm_brain_mask_pt at 0x7f5ac68aaf80>),
  'notl_brain_ct': functools.partial(<function create_mr_ct_dsource at 0x7f5ac68b3f80>, '/home/turgutluk/data/ventricles_data/ct', '/home/turgutluk/data/ventricles_data/csvs/ct_splits_df.csv', <function tfm_image_pt at 0x7f5b396a6200>, <function tfm_brain_mask_pt at 0x7f5ac68aaf80>),
  'notl_ventricle_mr': functools.partial(<function create_mr_ct_dsource at 0x7f5ac68b3f80>, '/home/turgutluk/data/ventricles_data/mr', '/home/turgutluk/data/ventricles_data/csvs/mr_splits_df.csv', <function tfm_skull_stripped_image_pt at 0x7f5ac68b3dd0>, <function tfm_ventricles_mask_pt at 0x7f5ac68b3e60>),
  'notl_ventricle_ct': functools.partial(<function create_mr_ct_dsource at 0x7f5ac68b3f80>, '/home/turgutluk/data/ven

In [14]:
len(experiment_model_dict), experiment_model_dict # all possible model configs for experiments

(11,
 {'baseline1': functools.partial(<function unet_default at 0x7f5ac68b3d40>, p=0.0, norm_type='batch', actn='relu'),
  'baseline2': functools.partial(<function unet_default at 0x7f5ac68b3d40>, p=0.0, norm_type='batch', actn='relu'),
  'baseline3': functools.partial(<function unet_default at 0x7f5ac68b3d40>, p=0.0, norm_type='group', actn='relu'),
  'baseline4': functools.partial(<function unet_default at 0x7f5ac68b3d40>, p=0.0, norm_type='group', actn='prelu'),
  'baseline5': functools.partial(<function unet_default at 0x7f5ac68b3d40>, p=0.3, norm_type='group', actn='prelu'),
  'baseline6': <function local.models.meshnet()>,
  'baseline7': functools.partial(<function unet_wide at 0x7f5ac684bef0>, p=0.0, norm_type='group', actn='prelu'),
  'baseline8': functools.partial(<function unet_deep at 0x7f5ac684bf80>, p=0.0, norm_type='group', actn='prelu'),
  'baseline9': functools.partial(<function unet_wide_deep at 0x7f5ac684e050>, p=0.0, norm_type='group', actn='prelu'),
  'baseline10': 

### args for experiments

In [9]:
gpu=0

data_name = "notl_brain_mr"
sample_size = 10
seed=42
bs=2
model_name='baseline1'
MODEL_NAME='TL_Brain_MR_Baseline_1'
model_dir='tl_brain_mr_models'
loss_func='dice'

TASK='BRAIN'
MODALITY='MR'
tl_model_path="atlas_brain_mr_models/ATLAS_Brain_MR_Baseline_1" # relative path

eps=1e-8
epochs=2
lr=0.1

### 1) data

In [34]:
#export
@contextmanager
def np_local_seed(seed):
    "numpy local seed - doesn't effect global random state"
    state = np.random.get_state()
    np.random.seed(seed)
    try: yield
    finally: np.random.set_state(state)

In [35]:
dsource = datasource_dict[data_name]()

In [36]:
if sample_size:
    with np_local_seed(seed):
        dsource.splits[0] = L(np.random.choice(dsource.splits[0], sample_size))  # subsample training

In [37]:
dsource.splits[0]

array([131,  68, 118,  20, 135,  91,  80,  28, 131, 106])

In [38]:
dbunch = dsource.databunch(after_batch=[Cuda()], bs=bs)

In [39]:
dbunch.dls

(<fastai2.data.core.TfmdDL at 0x7f3669b3ec50>,
 <fastai2.data.core.TfmdDL at 0x7f36688f5190>)

In [40]:
# create for test1 and test2 dls
if len(dbunch.dls) == 4: pass
elif len(dbunch.dls) == 2: pass
else: raise Exception(f"DataSource should have either 2 or 4 subsets, but have {len(dsource.splits)}")

### model 

In [41]:
from fastai2.torch_core import *
from fastai2.basics import *
from local.trainutils import *

In [42]:
# model
m = experiment_model_dict[model_name]()
apply_leaf(m, partial(my_cond_init, func=nn.init.kaiming_normal_))

### callbacks 

> There is an issue with CSVLogger - not able to create directory when not existing

> SaveModelCallback have problem with loading with distributed

In [43]:
from fastai2.callback.all import *

In [44]:
#export
class SaveModelCallback(TrackerCallback):
    "A `TrackerCallback` that saves the model's best during training and loads it at the end."
    def __init__(self, monitor='valid_loss', comp=None, min_delta=0., fname='model', every_epoch=False, add_save=None, with_opt=False):
        super().__init__(monitor=monitor, comp=comp, min_delta=min_delta)
        store_attr(self, 'fname,every_epoch,add_save,with_opt')

    def _save(self, name):
        self.learn.save(name, with_opt=self.with_opt)
        if self.add_save is not None:
            with self.add_save.open('wb') as f: self.learn.save(f, with_opt=self.with_opt)

    def after_epoch(self):
        "Compare the value monitored to its best score and save if best."
        if self.every_epoch: self._save(f'{self.fname}_{self.epoch}')
        else: #every improvement
            super().after_epoch()
            if self.new_best: self._save(f'{self.fname}')

# loading is a problem in distributed
#     def on_train_end(self, **kwargs):
#         "Load the best model."
#         if not self.every_epoch: self.learn.load(f'{self.fname}')

In [45]:
save_model_cb = SaveModelCallback( monitor='dice_score', comp=np.greater, every_epoch=False,
                        fname=f'best_of_{MODEL_NAME}')
# csv_logger_cb = CSVLogger(fname=f'{experiment_id}_history.csv')

In [46]:
callbacks = [TerminateOnNaNCallback(), save_model_cb]    

### learner

> Best models are saved under `experiments/{model_dir}/{MODEL_NAME}` directories

In [47]:
model_dir, MODEL_NAME

('tl_brain_mr_models', 'TL_Brain_MR_Baseline_1')

In [48]:
split_func = model_split_dict[model_name]

In [49]:
lf = loss_dict[loss_func]
opt_func = partial(Adam, eps=eps)
learn = Learner(dbunch, m, lf, metrics=[dice_score], opt_func=opt_func,
                path=Path('experiments'),
                model_dir=Path(model_dir)/MODEL_NAME, cbs=callbacks, splitter=split_func)
learn.to_fp16()

In [50]:
learn.path, learn.model_dir

(PosixPath('experiments'),
 PosixPath('tl_brain_mr_models/TL_Brain_MR_Baseline_1'))

### load pretrained

In [51]:
import yaml
with open(os.environ.get('YAML_TL', 'transfer_learning.yaml')) as f: tl = yaml.load(f.read(), yaml.FullLoader)

In [52]:
path, model_dir = learn.path, learn.model_dir
tl_model_name = tl[TASK][MODALITY][MODEL_NAME]
learn.path, learn.model_dir = path, tl_model_path
learn.load(tl_model_name);

In [53]:
learn.path, learn.model_dir = path, model_dir
learn.path, learn.model_dir

(PosixPath('experiments'),
 PosixPath('tl_brain_mr_models/TL_Brain_MR_Baseline_1'))

### fine tuning

In [54]:
n_groups = len(learn.opt.param_groups)
tl_epochs = array([epochs]*n_groups)//np.power(2, array(list(range(n_groups))))
tl_epochs = np.clip(tl_epochs,1,np.inf).astype(int)

In [55]:
list(tl_epochs), list(range(n_groups))

([2, 1, 1], [0, 1, 2])

In [33]:
for i, _epochs in zip(range(1, n_groups+1), tl_epochs):
    if not int(gpu): print(f"Freezing to param group: {-i} and training for {_epochs} epochs")
    learn.freeze_to(-i)
    learn.fit_one_cycle(_epochs, slice(lr), cbs=callbacks)

Freezing to param group: -1 and training for 2 epochs


epoch,train_loss,valid_loss,dice_score,time
0,0.754202,0.761797,0.102429,00:32
1,0.730773,0.768464,0.115393,00:12


Freezing to param group: -2 and training for 1 epochs


epoch,train_loss,valid_loss,dice_score,time
0,0.652640,0.903330,0.051248,00:28


Freezing to param group: -3 and training for 1 epochs


epoch,train_loss,valid_loss,dice_score,time
0,0.544313,0.806283,0.138899,00:27


### evaluate

In [56]:
learn.load(f'best_of_{MODEL_NAME}');

In [57]:
learn.cbs = [cb for cb in learn.cbs if not isinstance(cb, TrackerCallback) and
                                       not isinstance(cb, TerminateOnNaNCallback)]
# learn.validate(1) # debug - best model is saved?

In [60]:
learn.model_dir

PosixPath('tl_brain_mr_models/TL_Brain_MR_Baseline_1')

In [62]:
from time import time

In [63]:
# save test1 and test2 results for unique experiment at time.time()
if len(dbunch.dls) == 4: 
    test1_eval, test2_eval = learn.validate(2), learn.validate(3)
    eval_dir = f"test_results/{model_dir}"
    os.makedirs(eval_dir, exist_ok=True)
    save_fn = f"{eval_dir}/{str(int(time()))}.txt"
    with open(save_fn, 'w') as f: f.write(str([test1_eval, test2_eval]))

In [64]:
learn.path

PosixPath('experiments')

In [65]:
model_dir, MODEL_NAME

(PosixPath('tl_brain_mr_models/TL_Brain_MR_Baseline_1'),
 'TL_Brain_MR_Baseline_1')

### `script` - Don't execute cells below!

These cells are converted into a python script in `training_scripts/traintransfer.py` which you can use instead of running in noteboook

In [19]:
data_name = "notl_brain_mr"
sample_size = 10
seed=42
bs=2
model_name='baseline1'
MODEL_NAME='TL_Brain_MR_Baseline_1'
model_dir='tl_brain_mr_models'
loss_func='dice'
TASK='BRAIN'
MODALITY='MR'
tl_model_path="atlas_brain_mr_models/ATLAS_Brain_MR_Baseline_1" # relative path
eps=1e-8
epochs=2
lr=0.1

In [ ]:
#export 
@call_parse
def main(
    gpu:Param("GPU to run on", str)=None,
    data_name:Param("Data name for experiment", str)="notl_brain_mr",
    sample_size:Param("Random samples for training, default None - full", int)=None,
    seed:Param("Random seed for sample_size", int)=None,
    bs:Param("Batch size for training", int)=2,
    model_name:Param("Model architecture config - baseline*", str)="baseline1",
    MODEL_NAME:Param("Model name to save the model", str)="TL_Brain_MR_Baseline_1",
    model_dir:Param("Directory to save model", str)="tl_brain_mr_models",
    loss_func:Param("Loss function for training", str)='dice',
    TASK:Param("Task defined for transfer learning in tl.yaml", str)='BRAIN',
    MODALITY:Param("Modality defined for transfer learning in tl.yaml", str)='MR',
    tl_model_path:Param("Relative model path", str)="atlas_brain_mr_models/ATLAS_Brain_MR_Baseline_1", 
    eps:Param("Eps value for Adam optimizer", float)=1e-8,
    epochs:Param("Number of epochs for training", int)=2,
    lr:Param("Learning rate for training", float)=0.1):
    
    "Distributed de novo training - aka. from scratch"
    import os; print(os.getcwd())

    gpu = setup_distrib(gpu)
    n_gpus, gpu_rank = num_distrib(), rank_distrib()

    # data
    dsource = datasource_dict[data_name]()
    if sample_size:
        with np_local_seed(seed):
            dsource.splits[0] = L(np.random.choice(dsource.splits[0], sample_size))  # subsample training
    dbunch = dsource.databunch(after_batch=[Cuda()], bs=bs)
    if len(dbunch.dls) == 4: pass
    elif len(dbunch.dls) == 2: pass
    else: raise Exception(f"DataSource should have either 2 or 4 subsets, but have {len(dsource.splits)}")

    # model
    m = experiment_model_dict[model_name]()
    apply_leaf(m, partial(my_cond_init, func=nn.init.kaiming_normal_))
    
    # callbacks
    save_model_cb = SaveModelCallback(monitor='dice_score', comp=np.greater, every_epoch=False,
                        fname=f'best_of_{MODEL_NAME}')
    callbacks = [TerminateOnNaNCallback(), save_model_cb]        
    
    # learn
    split_func = model_split_dict[model_name]
    lf = loss_dict[loss_func]
    opt_func = partial(Adam, eps=eps)
    learn = Learner(dbunch, m, lf, metrics=[dice_score], opt_func=opt_func,
                    path=Path('experiments'),
                    model_dir=Path(model_dir)/MODEL_NAME, cbs=callbacks, splitter=split_func)
    learn.to_fp16()

    # load pretrained
    with open(os.environ.get('YAML_TL', 'transfer_learning.yaml')) as f: 
        tl = yaml.load(f.read(), yaml.FullLoader)
    path, model_dir = learn.path, learn.model_dir
    tl_model_name = tl[TASK][MODALITY][MODEL_NAME]
    learn.path, learn.model_dir = path, tl_model_path
    learn.load(tl_model_name);
    learn.path, learn.model_dir = path, model_dir
    
    # distributed
    if gpu is None:       learn.to_parallel()
    elif num_distrib()>1: learn.to_distributed(gpu)    

    # fine tuning - transfer learning
    n_groups = len(learn.opt.param_groups)
    tl_epochs = array([epochs]*n_groups)//np.power(2, array(list(range(n_groups))))
    tl_epochs = np.clip(tl_epochs,1,np.inf).astype(int)
    for i, _epochs in zip(range(1, n_groups+1), tl_epochs):
        if not int(gpu): print(f"Freezing to param group: {-i} and training for {_epochs} epochs")
        learn.freeze_to(-i)
        learn.fit_one_cycle(_epochs, slice(lr), cbs=callbacks)
    


    # evaluate
    if not gpu_rank:
        if len(dbunch.dls) == 4: 
            learn.load(f'best_of_{MODEL_NAME}');
            learn.cbs = [cb for cb in learn.cbs if not isinstance(cb, TrackerCallback) and
                                               not isinstance(cb, TerminateOnNaNCallback)]        
            test1_eval, test2_eval = learn.validate(2), learn.validate(3)
            eval_dir = f"test_results/{model_dir}"
            os.makedirs(eval_dir, exist_ok=True)
            save_fn = f"{eval_dir}/{str(int(time()))}.txt"
            with open(save_fn, 'w') as f: f.write(str([test1_eval, test2_eval]))

### Export

In [36]:
# special export to fix for relative import problems
from local.notebook.export import *

In [37]:
def export_nb_to(src, dest):
    "export src notebook to dest"
    cells = read_nb(src)['cells']
    srcs = []
    for cell in cells: 
        src = cell['source']
        if src.startswith("#export"): srcs.append(src+"\n") 
        if src.startswith("# export"): srcs.append(src+"\n")
    res = "\n".join(srcs)
    with open(dest, "w") as f: f.write(res)

In [38]:
export_nb_to("3c) traintransfer.ipynb", "training_scripts/traintransfer.py")

###  `[notebook run #1]` - training in weakly supervised transfer learning mode

In [1]:
from fastai2.basics import *
import fastai2
from local.scriptrunner import *

launch_file = f"{Path(fastai2.__file__).parent}/launch.py"
script = "./training_scripts/traintransfer.py "

In [4]:
# distributed training with multiple gpus
# change parameters accordingly
cmd = f"""python {launch_file} \
--gpus=12 {script} \
--data_name=notl_brain_mr \
--sample_size=10 \
--seed=42 \
--bs=2 \
--model_name=baseline1 \
--MODEL_NAME=TL_Brain_MR_Baseline_1 \
--model_dir=tl_brain_mr_models \
--loss_func=bce \
--TASK=BRAIN \
--MODALITY=MR \
--tl_model_path=atlas_brain_mr_models/ATLAS_Brain_MR_Baseline_1 \
--eps=1e-8 \
--epochs=2 \
--lr=0.1"""

In [32]:
# single gpu training 
# change parameters accordingly
cmd = f"""python {script} \
--gpu=0 \
--data_name=notl_brain_mr \
--sample_size=10 \
--seed=42 \
--bs=2 \
--model_name=baseline1 \
--MODEL_NAME=TL_Brain_MR_Baseline_1 \
--model_dir=tl_brain_mr_models \
--loss_func=bce \
--TASK=BRAIN \
--MODALITY=MR \
--tl_model_path=atlas_brain_mr_models/ATLAS_Brain_MR_Baseline_1 \
--eps=1e-8 \
--epochs=2 \
--lr=0.1"""

In [5]:
cmd

'python /home/turgutluk/fastai2_fork/fastai2/launch.py --gpus=12 ./training_scripts/traintransfer.py  --data_name=notl_brain_mr --sample_size=10 --seed=42 --bs=2 --model_name=baseline1 --MODEL_NAME=TL_Brain_MR_Baseline_1 --model_dir=tl_brain_mr_models --loss_func=bce --TASK=BRAIN --MODALITY=MR --tl_model_path=atlas_brain_mr_models/ATLAS_Brain_MR_Baseline_1 --eps=1e-8 --epochs=2 --lr=0.1'

In [6]:
run_command(cmd)

Traceback (most recent call last):
  File "./training_scripts/traintransfer.py", line 11, in <module>
    from local.datasource import *
  File "/home/turgutluk/Vent_Seg_Project/dev/local/datasource.py", line 16, in <module>
    with open(os.environ.get('YAML_DATA', '../data.yaml')) as f: data_config = yaml.load(f.read(), yaml.FullLoader)
FileNotFoundError: [Errno 2] No such file or directory: '../data.yaml'
Traceback (most recent call last):
  File "./training_scripts/traintransfer.py", line 11, in <module>
    from local.datasource import *
  File "/home/turgutluk/Vent_Seg_Project/dev/local/datasource.py", line 16, in <module>
    with open(os.environ.get('YAML_DATA', '../data.yaml')) as f: data_config = yaml.load(f.read(), yaml.FullLoader)
FileNotFoundError: [Errno 2] No such file or directory: '../data.yaml'



0

### fin